# Intro to Julia

Julia is a high-level mathematical programming language that is flexible and fast. There are many powerful packages available in the ecosystem, but for our purposes we are mainly interested in modelling chemical reaction networks and symbolic computational algebra.

Below is a brief summary of the basic features of Julia, and key difference with other popular languages.

In [45]:
#TODO
#demo code showing whats possible

### Structure
- Installing
- Julia syntax
- Differences with other languages
- Simple Catalyst example with plots
- helpful resources

Other packages will need to be downloaded first. This can be done with the following code, or in the `Pkg` environment of the Julia terminal.

In [ ]:
#TODO installing packages
using Pkg
Pkg.activate(".")

Pkg.add("Symbolics")
Pkg.add("Plots")

## Basic usage

Julia's syntax is based on similar modern languages such as MATLAB, R, and Python.

In [ ]:
# Numeric Types
a = 1  # Integer
b = 2//3  # Rational
c = 4.0  # Floating point ("Real")
d = 5 + 6im  # Complex

true

In [48]:
# Type hierarchy
typeof(a) == Int
typeof(c) == Float64

b isa Rational
c isa Real

supertypes(Int)

(Int64, Signed, Integer, Real, Number, Any)

In [ ]:
using LinearAlgebra  # TODO

# Collections
v = [1.0, 3, 5//9]  # Vector
M = [
    1 2 3  
    4 5 6
    7 8 9
]  # Matrix

x = M * v
println(x)
x[1]  # Arrays are 1-indexed

[8.666666666666666, 22.333333333333332, 36.0]


8.666666666666666

In [ ]:
# Loops
for i in 1:2:9  # from 1 to 9 with stepsize = 2
    println(i^2, " ")
end

dict = Dict("key_1" => "value_1", "key_2" => "value_2")
for (key, val) in dict
    println("$key -> $val")
end

x = 1
while x <= 100
    x *= x + 1
    print(x, " ")
end

1 
9 
25 
49 
81 
key_1 -> value_1
key_2 -> value_2
2 6 42 1806 

In [17]:
# Control flow
(a > 1) && (b < 10) # AND
(a > 1) || (b < 10) # OR
!(a + b > 10) # NOT

if a > b
    println("$a is larger than $b")
elseif c >= b
    println("$c is larger than $b")
else
    println("Something else")
end

1 is larger than 2//3


In [37]:
# Function definitions
function myfunc(input)
    output = input^3
    output  # the last line of a function be returned (if not returned already)
end

println(myfunc(555))

# inline function definitions
f(x) = 3x^2 + 2x + 1
g(y) = 1 / (1 - y)

g(f(2))

170953875


-0.0625

Most functions have documentation included which can be accessed with the `@doc` macro e.g. `@doc abs`

In [68]:
# example help

## Julia specific features
The official [docs](https://docs.julialang.org/en/v1/manual/noteworthy-differences/) has a page on noteworthy differences with other languages

### Broadcasting
Many functions and operations can be *broadcast* over an array or other collection. This means the function is applied to each element of a list in a computationally efficient way. (This is similar to vectorised code in other languages.)

Broadcasted functions have a `.` before the operation or function call (those familiar with MATLAB will recognise this notation).

In [39]:
values = [2, 3, 5] .+ 10
println(values)

theta = 0:(pi/4):2pi
x = sin.(theta) # note the '.' before the '('
println(x)

# works for any function we define
x_list = 1:100
cube_list = myfunc.(x_list)
sum(cube_list)

[12, 13, 15]
[0.0, 0.7071067811865475, 1.0, 0.7071067811865476, 1.2246467991473532e-16, -0.7071067811865475, -1.0, -0.7071067811865477, -2.4492935982947064e-16]


25502500

### Macros
Julia has several in-built *macros* that perform meta-programming (code which writes code). Macros begin with an `@` sign.

Macros are commonly used in symbolic or modelling packages, including in Catalyst and Symbolics.

In [62]:
x = 1:5
@show collect(x)
@show x .^ 2
;

collect(x) = [1, 2, 3, 4, 5]
x .^ 2 = [1, 4, 9, 16, 25]


Notice that the printed output of `@show` included both the code itself and the calculated output. That is the essence of macros; they use the *code itself* as input data.

### Symbolics

In [ ]:
using Symbolics
@variables x y z

3-element Vector{Num}:
 x
 y
 z

In [ ]:
x isa Number  # true

f(t) = t^2 + t
sym_vec = f.([2x, y^2 + 1, x + z + y])

expand.(sym_vec)

3-element Vector{Num}:
                          (2//1)*x + (4//1)*(x^2)
                                 2 + 3(y^2) + y^4
 x + y + z + x^2 + 2x*y + 2x*z + y^2 + 2y*z + z^2

#### Compilation time
Code is compiled the first time Julia code is used in a session. This means that the first time code is run can appear slow, but each subsequent call will be much faster.

In [67]:
# Run the following twice
rand(100, 100)^3.5

100×100 Matrix{ComplexF64}:
 8800.35-3.34802im   8163.07-0.840518im   …  8550.26+0.337513im
 9494.85-0.868717im  8809.51-0.562227im      9226.97-0.21169im
  9438.0+2.52184im   8754.99+0.934599im      9172.55+0.00795573im
 8892.01-2.40154im   8255.41-0.357898im      8641.41+0.45516im
 9133.35+0.289228im  8477.36-0.0933768im     8885.46-0.173502im
 8940.61-1.4618im    8310.74-0.351798im   …  8698.25+0.160571im
 9811.74+3.19611im   9104.44+0.327347im      9537.64-0.735295im
 8350.42-0.58355im   7748.36-0.205168im      8100.79+0.00780885im
 9415.19+3.90789im   8743.17+1.04311im       9146.48-0.34001im
 8051.35-4.30718im   7473.04-1.25216im       7831.45+0.285635im
        ⋮                                 ⋱  
 8270.98+1.92588im    7675.1+0.894243im      8036.92+0.163048im
 9217.51+0.774756im  8558.85+0.327805im      8966.82+0.0378192im
 8809.54+1.61847im   8173.89+0.11349im       8562.93-0.417804im
 9539.63-4.65388im    8850.1-0.94282im       9267.09+0.665285im
 9175.86+0.393243im  8506.76

This should be enough to get started using Julia.

Continue to [Intro to Catalyst](Tutorial2.ipynb)